## Chain
1. Take natural language query from
2. Translate (using code LLM) the natural language query from #1 into SQL command
3. Run the query from #2 against the db

# Import project dependencies

In [145]:
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain.sql_database import SQLDatabase
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.agent_toolkits import create_sql_agent
from functools import partial
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import pandas as pd

import langchain
# langchain.verbose=True

import sqlglot
import os

### Pull database into project

In [146]:
db=SQLDatabase.from_uri("sqlite:///nba_stats.db")
print(db.get_table_info())


CREATE TABLE player_stats (
	"YEAR" TEXT, 
	"SEASON_TYPE" TEXT, 
	"PLAYER_RANK" INTEGER, 
	"PLAYER_NAME" TEXT, 
	"TEAM_ABRV" TEXT, 
	"GAMES_PLAYED" INTEGER, 
	"MINUTES_PLAYED" INTEGER, 
	"FIELD_GOALS_MADE" INTEGER, 
	"FIELD_GOALS_ATTEMPTED" INTEGER, 
	"FIELD_GOAL_PERCENTAGE" REAL, 
	"THREE_POINTERS_MADE" INTEGER, 
	"THREE_POINTERS_ATTEMPTED" INTEGER, 
	"THREE_POINT_PERCENTAGE" REAL, 
	"FREE_THROWS_MADE" INTEGER, 
	"FREE_THROWS_ATTEMPTED" INTEGER, 
	"FREE_THROW_PERCENTAGE" REAL, 
	"OFFENSIVE_REBOUNDS" INTEGER, 
	"DEFENSIVE_REBOUNDS" INTEGER, 
	"TOTAL_REBOUNDS" INTEGER, 
	"ASSISTS" INTEGER, 
	"STEALS" INTEGER, 
	"BLOCKS" INTEGER, 
	"TURNOVERS" INTEGER, 
	"PERSONAL_FOULS" INTEGER, 
	"POINTS" INTEGER, 
	"ASSISTS_TO_TURNOVER_RATIO" REAL, 
	"STEALS_TO_TURNOVER_RATIO" REAL, 
	"TEAM" TEXT
)

/*
3 rows from player_stats table:
YEAR	SEASON_TYPE	PLAYER_RANK	PLAYER_NAME	TEAM_ABRV	GAMES_PLAYED	MINUTES_PLAYED	FIELD_GOALS_MADE	FIELD_GOALS_ATTEMPTED	FIELD_GOAL_PERCENTAGE	THREE_POINTERS_MADE	THREE_POI

In [147]:
load_dotenv('secret.env')
api_key = os.getenv("NVIDIA_API_KEY")

In [148]:
def get_schema(_):
    return db.get_table_info()

### Create functions that will be used as chain inputs

In [149]:
def run_sql(query: str, dialect: str) -> str:
    """
    Transpile and execute an SQL query using the specified dialect.
    Parameters:
    - query (str): The SQL query to be transpiled and executed.
    - dialect (str): The target SQL dialect for transpilation and execution.
    Returns:
    str: The result of executing the transpiled SQL query.
    Example:
    ```
    result = run_sql("SELECT * FROM my_table", "sqlite")
    print(result)
    ```
    """
    query=sqlglot.transpile(query, read="postgres", write=dialect)[0]
    print(query)
    return db.run(query)

In [ ]:
# from langchain_nvidia_ai_endpoints import ChatNVIDIA

### Create a instance of ChatNVIDIA model name -- `mistralai/mamba-codestral-7b-v0.1`

In [152]:
sql_llm=ChatNVIDIA(
    model="mistralai/mamba-codestral-7b-v0.1",
    max_tokens=500
)

In [153]:
chat_llm=ChatNVIDIA(
    model = "meta/llama3-70b-instruct",
    max_tokens=1000
)

### Prepare SQL Prompt Template {input: question, schema}

In [211]:
SQL_GEN_TEMPLATE="""
### Task
Generate a SQLite query to answer. Always use ILIKE on TEXT columns [QUESTION]{question}[/QUESTION]
### Instructions
- Always provide all the columns, If you cannot answer the question with the available database schema, return 'I do not know'
### Database Schema
The query will run on a database with the following schema:
{schema}
### Answer
Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
[SUFFIX][/SQL][PREFIX]▁[SQL]\n
"""

### Create and invoke chain - save results in variable `response`

In [212]:
prompt=PromptTemplate.from_template(SQL_GEN_TEMPLATE)
sql_response=(
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | sql_llm.bind(stop=["[/SQL]", ";"])
    | StrOutputParser()
)
response=sql_response.invoke({"question": "What is Kobe Bryant's shooting percentage?"})
print(response)

SELECT FIELD_GOAL_PERCENTAGE, THREE_POINT_PERCENTAGE
FROM player_stats
WHERE PLAYER_NAME = 'Kobe Bryant'

GROUP BY SEASON_TYPE

[SQLITE]


### Create a `SystemMessageTemplate` and a `HumanMessageTemplate` to use for the `ChatPromptTemplate`

In [214]:
SQL_SUM_SYS_MSG=(
    "Given an input question and SQL response, convert it to a natural language answer."
    " Give a human like response"
)
SQL_SUM_TEMPLATE="""
Based on the table schema below, question, sql query, and sql response,\
 write a natural language response. Be generic and avoid errors as much as possible.
{schema}
Question: {question}
SQL Query: {sqlquery}
SQL Response: {response}
""".strip()

### Create a new chain `full_chain`, passing the previous outcome from `sql_chain` in as the first input
This is a crucial step where the model uses a RunnablePassthrough.assign tool to pass through the response from the sql_chain as the first input to the full_chain. 
This is where the natural language input is converted to SQL code to retrieve relevant information to answer the user questions which is the     

**Final Output: natural language response from ai model**

In [215]:
prompt_response=ChatPromptTemplate.from_messages([
    ("system", SQL_SUM_SYS_MSG),
    ("human", SQL_SUM_TEMPLATE),
])
history=[]
def save_return(d, history=[]):
    history+=[d]
    return d
full_chain=(
    RunnablePassthrough.assign(sqlquery=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_sql(x["sqlquery"], "sqlite") or "ERROR RETRIEVING FROM DATABASE",
    )
    | prompt_response
    | partial(save_return, history=history)
    | chat_llm
    | StrOutputParser()
)

# GRADIO FRONT-END
We want to build a Gradio front-end that:

Takes a natural language query from the user.
Translates it into an SQL query using Llama LLM.
Executes the SQL query against your database and shows the results.
I'll guide you in setting up this interface.

Overview of What We’ll Do
Install Required Libraries.
Set Up Environment Variables.
Write the Gradio Interface Code.
Run and Test the Gradio Interface.


In [218]:
# pip install gradio langchain llama-index python-dotenv


In [219]:
# import gradio as gr
# from langchain_core.runnables import RunnablePassthrough
# from functools import partial
# import sqlglot
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# from langchain.sql_database import SQLDatabase
# import os
# import re
# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser


# # Load environment variables
# from dotenv import load_dotenv
# load_dotenv('secret.env')
# api_key = os.getenv("NVIDIA_API_KEY")

# # Initialize SQL LLM and Database
# sql_llm = ChatNVIDIA(
#     model="mistralai/mamba-codestral-7b-v0.1",
#     max_tokens=500,
#     api_key=api_key
# )

# db = SQLDatabase.from_uri("sqlite:///nba_stats.db")



# # Define the database schema and SQL generation functions

# def get_schema(_):
#     return db.get_table_info()

# def clean_sql_response(response: str) -> str:
#     """
#     Clean up the LLM response to extract only the SQL query.
#     """
#     # Enhance regex to capture SQL queries more effectively, even if they contain multiple lines
#     sql_match = re.search(r"(SELECT|INSERT|UPDATE|DELETE|WITH|CREATE|DROP|ALTER)\s.*?(;|$)", response, re.IGNORECASE | re.DOTALL)
#     if sql_match:
#         return sql_match.group(0).strip()
#     return ""

# def run_sql(query: str, dialect: str) -> str:
#     try:
#         if not query.strip():
#             return "Error: Received an empty SQL query."
#         # Modify the transpilation process to ensure compatibility
#         transpiled_queries = sqlglot.transpile(query, read="postgres", write=dialect)
#         if not transpiled_queries:
#             return "Error: Failed to transpile the SQL query."
#         transpiled_query = transpiled_queries[0]
#         print(f"Transpiled SQL Query: {transpiled_query}")  # Debugging line to check transpiled query
#         result = db.run(transpiled_query)
#         return str(result)
#     except Exception as e:
#         return f"Error executing SQL query: {str(e)}"

# # Full chain approach for generating SQL
# SQL_GEN_TEMPLATE = """
# ### Task
# Generate a SQLite query to answer, but use the LIMIT clause to limit the output to 5 results. Always use ILIKE on TEXT columns [QUESTION]{question}[/QUESTION]
# ### Instructions
# - Always provide all the columns. If you cannot answer the question with the available database schema, return 'I do not know'
# ### Database Schema
# The query will run on a database with the following schema:
# {schema}
# ### Answer
# Given the database schema, here is the SQLite query that answers [QUESTION]{question}[/QUESTION]
# """

# prompt_template = PromptTemplate.from_template(SQL_GEN_TEMPLATE)

# prompt_response=ChatPromptTemplate.from_messages([
#     ("system", SQL_SUM_SYS_MSG),
#     ("human", SQL_SUM_TEMPLATE),
# ])

# history=[]

# def save_return(d, history=[]):
#     history+=[d]
#     return d

# # {'key': 'value'}
# #

# def clean_sql(sql_response):
#     print(f"Generated SQL Response: {sql_response['sqlquery']}")  # Debugging line to check LLM response
#     cleaned_sql = clean_sql_response(sql_response['sqlquery'])
#     sql_response['sqlquery']=cleaned_sql
#     print(f"Cleaned SQL Query: {cleaned_sql}")  # Debugging line to check cleaned SQL query
#     if not cleaned_sql:
#         return "Error: Generated SQL query is invalid or could not be extracted. Please ensure your question is specific and structured to request data."
#     return sql_response

# def ask_question_with_full_chain(question):
#     try:
#         history=[]
#         sql_response = (
#             RunnablePassthrough.assign(schema=get_schema)
#             | prompt_template
#             | sql_llm.bind(stop=["[/SQL]", ";"])
#             | StrOutputParser()
#         )

#         full_chain=(
#             RunnablePassthrough.assign(sqlquery=sql_response)
#             | clean_sql
#             | RunnablePassthrough.assign(
#                 schema=get_schema,
#                 response=lambda x: run_sql(x["sqlquery"], "sqlite") or "ERROR RETRIEVING FROM DATABASE",
#             )
#             | prompt_response # sqlquery
#             | partial(save_return, history=history)
#             | chat_llm
#             | StrOutputParser()
#         )

#         response=full_chain.invoke({'question': question})
#         return response
#     # try:
#     #     sql_response = (
#     #         RunnablePassthrough.assign(schema=get_schema)
#     #         | prompt_template
#     #         | sql_llm.bind(stop=["[/SQL]", ";"])
#     #         | StrOutputParser()
#     #     ).invoke({"question": question})

#     #     print(f"Generated SQL Response: {sql_response}")  # Debugging line to check LLM response
#     #     cleaned_sql = clean_sql_response(sql_response)
#     #     print(f"Cleaned SQL Query: {cleaned_sql}")  # Debugging line to check cleaned SQL query
#     #     if not cleaned_sql:
#     #         return "Error: Generated SQL query is invalid or could not be extracted. Please ensure your question is specific and structured to request data."
#     #     response = run_sql(cleaned_sql, 'sqlite')
#     #     return response
#     except Exception as e:
#         return f"Error generating response: {str(e)}"

# # Define Gradio interface elements
# def chat(query):
#     response = ask_question_with_full_chain(query)
#     return response

# # # Create Gradio UI
# # demo = gr.Interface(
# #     fn=chat,
# #     inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
# #     outputs=gr.Textbox(),
# #     title="Comprehensive NBA DataBase Stat-Searcher",
# #     description="Ask questions in natural language, and get responses from the database using SQL."
# # )

# # # Launch the Gradio application
# # demo.launch()


# Model Validation

Validation Process Steps:
1. Get responses to `nba_questions`
- Run nba_questions through `full_chain` using `batch`
2. Create `PromptTemplate` {input: question, answer}
3. Get access to LLM
- Create LLM instance
4. Get access to `StringOuputParser`
- Create StrOutputParser() instance
5. Combine question & answer, create and invoke chain {chain: chat_template_validation | chat_llm | string_parser - output - : 0 'relevant' or 1 'irrelevant' validation}
6. Visualize data in DataFrame
- Create A DataFrame
7. Create function to calculate relevancy score (relevant / total)


## Step 1: Get responses to `nba_questions`

In [220]:
nba_questions = [
    # Predefined questions structured to get a response from model
    {"question": "Who had the highest assists-to-turnover ratio in 2020?"},
    {"question": "Which player scored the most points for the Golden State Warriors in 2016?"},
    {"question": "How many games did Giannis Antetokounmpo play in 2019?"},
    {"question": "Which player had the most defensive rebounds for the Miami Heat in 2021?"},
    {"question": "What was the field goal percentage of Stephen Curry in 2017?"},
    {"question": "Who recorded the most steals for the Los Angeles Lakers in 2022?"},
    {"question": "Which player made the most three-pointers in 2023?"},
    {"question": "How many blocks did Anthony Davis have in 2018?"},
    {"question": "Which player had the most free throws made for the Boston Celtics in 2015?"},
    {"question": "Who had the highest three-point percentage for the Dallas Mavericks in 2018?"},
    # 5 Random Edgde Case Questions To Show Real-Life Scenarios Into Model Validation Score
    # Team Questions
    {"question": "Which player has the most steals for the Knicks in 2016?"},
    {"question": "Who is the best rebounder for the Bulls in 2014?"},
    # Player Questions
    {"question": "Who had the most assists in the 2020?"},
    # Valid Queries on Player
    {"question": "Who is the top scorer in the 2020 season?"},
    # Out of Scope
    {"question": "How many championships have the Lakers won?"}
]

### - Run `nba_questions` through full_chain using `batch`

In [222]:
try:
  responses = full_chain.batch(nba_questions)
except Exception as e:
  print(f"Error generating responses: {str(e)}")

SELECT player_stats.PLAYER_NAME, SUM(player_stats.POINTS) AS total_points FROM player_stats JOIN teams ON player_stats.TEAM = teams.TEAM WHERE player_stats.YEAR = '2016' AND teams.TEAM_ABRV = 'GSW' GROUP BY player_stats.PLAYER_NAME ORDER BY total_points DESC NULLS FIRST LIMIT 1
Error generating responses: Invalid expression / Unexpected token. Line 1, Col: 11.
  To find the player with the highest assists-to-turnover ratio in 2020, you would need to join multiple tables s


## Step 2: Combine the question & answers to format properly for `PromptTemplate`

In [224]:
# Combine the questions and answers
combined_data = [{"question": q["question"], "answer": a} for q, a in zip(nba_questions, responses)]

# Print the combined list
for item in combined_data:
    print(item)

{'question': 'Who had the highest assists-to-turnover ratio in 2020?', 'answer': 'The player with the highest assists-to-turnover ratio in 2020 is Dakota Mathias, with a remarkable ratio of 13.'}
{'question': 'Which player scored the most points for the Golden State Warriors in 2016?', 'answer': 'I apologize, but there is no data available for the Golden State Warriors in 2016 in the provided tables. The tables only contain data for the 2012 season and do not include the Golden State Warriors. Therefore, it is not possible to determine which player scored the most points for the Golden State Warriors in 2016.'}
{'question': 'How many games did Giannis Antetokounmpo play in 2019?', 'answer': 'I apologize, but there is no data available for Giannis Antetokounmpo in 2019 in the provided tables. The query returned an error because Giannis Antetokounmpo is not present in the provided data.'}
{'question': 'Which player had the most defensive rebounds for the Miami Heat in 2021?', 'answer': '

## Step 3: Create PromptTemplate {input: question, answer}

In [225]:
chat_template_validation = ChatPromptTemplate.from_template(
    '''
    Analyze a question [QUESTION]{question}[/QUESTION] and answer [ANSWER]{answer}[/ANSWER] pair.
    Answer using a binary response of 0 = which means the answer is 'relevant', 1 = 'irrelevant' meaning the answer was irrelevant, NO OTHER WORDS, just 0 or 1.
    '''
)

In [226]:
chat_template_validation

ChatPromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template="\n    Analyze a question [QUESTION]{question}[/QUESTION] and answer [ANSWER]{answer}[/ANSWER] pair.\n    Answer using a binary response of 0 = which means the answer is 'relevant', 1 = 'irrelevant' meaning the answer was irrelevant, NO OTHER WORDS, just 0 or 1.\n    "), additional_kwargs={})])

## Step 4: Get access to LLM

In [227]:
chat_llm=ChatNVIDIA(
   model = "meta/llama3-70b-instruct",
   model_kwargs = {'seed': 365},
   temperature = 0,
   max_tokens=100
)

## Step 5: Get access to StringOuputParser

In [228]:
string_parser = StrOutputParser()

## Step 6: Combine question & answer, create and invoke chain {chain: chat_template_validation | chat_llm | string_parser - output - : 0 'relevant' or 1 'irrelevant' validation}

In [229]:
# Ensure combined_data is passed correctly into the chain
# Wrap combined_data in a dict or handle it in a way expected by the chain
processed_data = []
for data in combined_data:
    input_data = {"question": data["question"], "answer": data["answer"]}

    # Process each item in the combined_data through the chain
    try:
        result = chat_template_validation | chat_llm | string_parser
        output = result.invoke(input_data)  # Pass as dictionary
        print(output)  # Print or handle the chain's output
        processed_data.append({"question": data["question"], "answer": data["answer"], "validation": output})
    except Exception as e:
        print(f"Error processing input {input_data}: {str(e)}")

0
1
1
0
1
0
0
0
0
0


## Step 7: Visualize data in DataFrame

In [ ]:
validation_df = pd.DataFrame(processed_data)
validation_df.head()

,question,answer,validation
0,Who had the highest assists-to-turnover ratio ...,The player with the highest assists-to-turnove...,0
1,Which player scored the most points for the Go...,"I apologize, but there is no data available fo...",1
2,How many games did Giannis Antetokounmpo play ...,"I apologize, but there is no data available fo...",1
3,Which player had the most defensive rebounds f...,"""In 2021, Bam Adebayo led the Miami Heat with ...",0
4,What was the field goal percentage of Stephen ...,"I apologize, but there is no data available fo...",1
5,Who recorded the most steals for the Los Angel...,"Based on the data, Jarred Vanderbilt recorded ...",0
6,Which player made the most three-pointers in 2...,"In the 2023 regular season, Stephen Curry led ...",0
7,How many blocks did Anthony Davis have in 2018?,Anthony Davis had 135 blocks in 2018.,0
8,Which player had the most free throws made for...,"Here is a natural language response:\n\n""In 20...",0
9,Who had the highest three-point percentage for...,"I apologize, but there is no data available fo...",0


In [232]:
# Count the number of 0s in the 'games_played' column
zero_count = validation_df['validation'].value_counts()
zero_count

,count
validation,
0,7
1,3


## Step 8: Create function to calculate relevancy score (relevant / total)

In [233]:
def calculate_zero_score(df, column_name):
    zero_count = (df[column_name] == '0').sum()  # Count of 0's in the column
    total_rows = len(df)  # Total number of rows
    score = (zero_count / total_rows)*100  # Score as a fraction
    return score

# Calculate the score
score = calculate_zero_score(validation_df, "validation")
print(f"Model Relevancy Score - (0's (for relevant)/total rows) for 'validation - ': {score:.2f}%")

Model Relevancy Score - (0's (for relevant)/total rows) for 'validation - ': 70.00%


# 70% Relevance
This is not an accurate reflection of the models true relevance. The questions are guided to retrieve an actual response from the model, take a look at our edge cases questions (the last 5 questions) where the model has enormous room for improvement.